In [14]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Step: Load data

In [15]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core import Workspace, Dataset, Datastore

# Assuming you have already set up your workspace
workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

In [16]:
environment = Environment.from_conda_specification("nfl_env", "nfl_env.yaml")

In [17]:
# Create a compute target
compute_name = "matthewhanauer991"
# Define the environment and dependencies for your script


try:
    compute_target = ComputeTarget(workspace=workspace, name=compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [18]:
# Create a ScriptRunConfig for your script
src = ScriptRunConfig(
    source_directory=".", script="production_injury_data.py", environment=environment
)

In [19]:
# Create a PythonScriptStep
step = PythonScriptStep(
    name="scrapper_nfl_injury",
    script_name="production_injury_data.py",
    compute_target=compute_target,
    runconfig=src.run_config,
)

In [20]:
# Create a Pipeline using the PythonScriptStep
pipeline = Pipeline(workspace=workspace, steps=[step])

In [21]:
# Publish the pipeline
published_pipeline = pipeline.publish(name="scrapper_nfl_injury_pipeline")

Created step scrapper_nfl_injury [fa092189][1410d314-ad98-4eca-ad82-00acff48ab0a], (This step will run and generate new outputs)


In [23]:
from azureml.core import Workspace

ws = Workspace.from_config()
compute_target = ws.compute_targets["matthewhanauer991"]
print(compute_target.status.serialize())

{'errors': [], 'creationTime': '2023-03-30T20:59:17.305757+00:00', 'createdBy': {'userObjectId': '41cc7223-ef73-4687-b24c-1b4ca1bee586', 'userTenantId': 'a93eafc8-c064-4a88-bc01-47e93068b42b', 'userName': None}, 'modifiedTime': '2023-05-14T13:31:38.000711+00:00', 'state': 'Stopped', 'vmSize': 'STANDARD_DS11_V2'}


In [22]:
from datetime import time


recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="14:24")
schedule = Schedule.create(
    workspace,
    name="scrapper_nfl_injury_schedule",
    pipeline_id=published_pipeline.id,
    experiment_name="scrapper_nfl_injury_experiment",
    recurrence=recurrence,
)